In [ ]:
%load_ext autoreload
%autoreload 2
from __future__ import print_function
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import os, cPickle
from collections import defaultdict

import numpy as np
import kde
import simplebinmi
import keras.backend as K

import utils
trn, tst = utils.get_mnist()

infoplane_measure = 'upper'
infoplane_measure = 'bin'

MAX_EPOCHS = 10000
#MAX_EPOCHS = 10
DO_SAVE=True
ARCH = '1024-20-20-20'
#ARCH = '20-20-20-20-20-20'
#ARCH = '32-28-24-20-16-12'
#ARCH = '32-28-24-20-16-12-8-8'
BASE_DIRS = [ (act, 'rawdata/%s_%s/'%(act,ARCH)) for act in ['relu','tanh'] ]
SKIP_RELU = False

labelixs = [ tst.y == i for i in range(10) ]
#BASE_DIRS = [ (act, 'rawdata/%s_128-64-32-16-16/'%act) for act in ['relu','tanh'] ]

#BASE_DIRS = [ (act, 'rawdata/%s_20-20-20-20-20-20/'%act) for act in ['relu','tanh'] ]
#BASE_DIRS = [ (act, 'rawdata/%s_50-20-20-20/'%act) for act in ['relu','tanh'] ]
#BASE_DIRS = [ (act, 'rawdata/%s_20-20-20-20/'%act) for act in ['relu','tanh'] ]
#BASE_DIRS = [ (act, 'rawdata/%s_100-50-20-20-20/'%act) for act in ['relu','tanh'] ]
#BASE_DIRS = [ (act, 'rawdata/%s_20-20-20-20-20/'%act) for act in ['relu','tanh'] ]
#BASE_DIRS = [ (act, 'rawdata/%s_100-50-20/'%act) for act in ['relu','tanh'] ]
#variance = 0.25 # 1e-2 #  2e-1
subsample = 1
#variance = 0.25e-1 
variance = 1e-1 

In [ ]:
measures = {}

In [ ]:
activityVariable = K.placeholder(ndim=2)
entropy_func_upper = K.function([activityVariable,], [kde.entropy_estimator_kl(activityVariable, variance),])
entropy_func_lower = K.function([activityVariable,], [kde.entropy_estimator_bd(activityVariable, variance),])

from scipy.special import gammaln, psi

#dist_func = K.function([activityVar,], [kde.Kget_dists(activityVar),])
def Kentropy2(X, k=1):
    if k != 1:
        raise Exception('k must be 1')
    dists = kde.Kget_dists(X)
    
    dims, N = kde.get_shape(X)
    dists2 = dists + kde.tensor_eye(K.cast(N, 'int32')) * 10e20

    nearest_neighbor = K.min(dists2, axis=1)
    '''
    F. Perez-Cruz, (2008). Estimation of Information Theoretic Measures
    for Continuous Random Variables. Advances in Neural Information
    Processing Systems 21 (NIPS). Vancouver (Canada), December.
 
    return d*mean(log(r))+log(volume_unit_ball)+log(n-1)-log(k)
    '''
    return dims*K.mean(K.log(nearest_neighbor))#  + log_volume_unit_ball + psi(N) - psi(k)
    #return d*np.mean(np.log(r))+log_volume_unit_ball+np.log(n-1)-np.log(k)

entropy2_func = K.function([activityVariable,], [Kentropy2(activityVariable,),])
def entropy2(X,k=1):
    N, dims = X.shape
    log_volume_unit_ball = 0.5*dims*np.log(np.pi) - gammaln(.5*dims + 1)
    val = entropy2_func([X,])[0]
    return val + log_volume_unit_ball + psi(N) - psi(k)


In [ ]:
DO_PLOT_LAYERS = None
DO_LOWER = False
#del measures['tanh'][1800]
#del measures['tanh'][1900]
#del measures['tanh'][2000]
#del measures['tanh'][20]


for activation, BASE_DIR in BASE_DIRS:
    if activation not in measures:
        measures[activation] = {}
    if SKIP_RELU and activation == 'relu':
        continue
    print('***************')
    if not os.path.exists(BASE_DIR):
        continue
    for epochfile in sorted(os.listdir(BASE_DIR)):
        if not epochfile.startswith('epoch'):
            continue

        epoch =  int(epochfile[len('epoch'):])
        
        if epoch in measures[activation]:
            continue
            
        if epoch > MAX_EPOCHS:
            continue

        print("Doing %s%s" %(BASE_DIR, epochfile))
        
        with open(BASE_DIR+ "/"+epochfile, 'rb') as f:
            d = cPickle.load(f)

        num_layers = len(d['data']['activity_tst'])

        if DO_PLOT_LAYERS is None:
            DO_PLOT_LAYERS = []
            for lndx in range(num_layers):
                #if d['data']['activity_tst'][lndx].shape[1] < 200 and lndx != num_layers - 1:
                DO_PLOT_LAYERS.append(lndx)
                
        cepochdata = defaultdict(list)
        for lndx in range(num_layers):
            activity = d['data']['activity_tst'][lndx][::subsample]
            if True: #  and lndx != num_layers - 1:
                # Not the last layer
                hM_given_X = kde.kde_condentropy(activity, variance)
                h_upper = entropy_func_upper([activity,])[0]
                if DO_LOWER:
                    h_lower = entropy_func_lower([activity,])[0]

                hM_given_Y_upper=0.
                hM_given_Y_lower=0.
                for i in range(10):
                    ixs = tst.y[::subsample] == i
                    hcond_upper = entropy_func_upper([activity[ixs,:],])[0]
                    prob = np.mean(ixs)
                    hM_given_Y_upper += prob * hcond_upper
                    if DO_LOWER:
                        hcond_lower = entropy_func_lower([activity[ixs,:],])[0]
                        hM_given_Y_lower += prob * hcond_lower
            else:
                # The last layer. Treat probabilistically
                ps = activity.mean(axis=0)
                h_lower = h_upper = sum([-p*np.log(p) for p in ps if p != 0])

                x = -activity * np.log(activity)
                x[activity == 0] = 0.
                hM_given_X = np.mean(x.sum(axis=1))

                hM_given_Y=0.
                for i in range(10):
                    ixs = tst.y[::subsample] == i
                    ps = activity[ixs,:].mean(axis=0)
                    hcond = sum([-p*np.log(p) for p in ps if p != 0])
                    prob = np.mean(ixs)
                    hM_given_Y += prob * hcond
                hM_given_Y_lower = hM_given_Y_upper = hM_given_Y
                del hM_given_Y
                
            iLN2 = 1.0/np.log(2) # nats to bits
        
            cepochdata['MI_XM_upper'].append( iLN2*(h_upper-hM_given_X) )
            cepochdata['MI_YM_upper'].append( iLN2*(h_upper-hM_given_Y_upper) )
            cepochdata['H_M_upper'].append(iLN2*h_upper)

            
            if DO_LOWER:
                cepochdata['MI_XM_lower'].append( iLN2*(h_lower-hM_given_X) )
                cepochdata['MI_YM_lower'].append( iLN2*(h_lower-hM_given_Y_lower) )
                cepochdata['H_M_lower'].append(iLN2*h_lower)

            #measures[activation][epoch]['MI_XM_bin'].append(simplebinmi.bin_calc_information(tst.X, activity, 5)
            binxm, binym = simplebinmi.bin_calc_information2(labelixs, activity, 0.5)
            cepochdata['MI_XM_bin'].append(iLN2 * binxm)
            cepochdata['MI_YM_bin'].append(iLN2 * binym)
            
            # h2 = entropy2(activity)
            # #jointH = entropy2(np.hstack([tst.X, activity]), nnK)
            # #mi2 = h2 + inputH - jointH
            # cepochdata['H_M2'].append(iLN2*h2)
            # #MI_XM2[epoch].append(mi2)

            #print('- Layer %d: MI(X;M)=%0.3f, MI(Y;M)=%0.3f,  MI(X;M)bin=%0.3f'
            #      %(lndx, lastd['MI_XM_upper'][-1], lastd['MI_YM_upper'][-1], lastd['MI_XM_bin'][-1]))
            print('- Layer %d upper: MI(X;M)=%0.3f, MI(Y;M)=%0.3f / '
                  %(lndx, cepochdata['MI_XM_upper'][-1], cepochdata['MI_YM_upper'][-1]),
                 'bin: MI(X;M)=%0.3f, MI(Y;M)=%0.3f'
                  %(cepochdata['MI_XM_bin'][-1], cepochdata['MI_YM_bin'][-1]))

        measures[activation][epoch] = cepochdata
        #print()

    

In [ ]:
#PLOT_LAYERS = [0,1,2,3,4] # [1,2,3]
#PLOT_LAYERS = [0,1,2,3]
#PLOT_LAYERS = [0,1,2,3]
plt.figure(figsize=(8,8))
gs = gridspec.GridSpec(4,2)
for actndx, (activation, _) in enumerate(BASE_DIRS):
    if activation not in measures:
        print("Skipping", activation)
        continue
    epochs = sorted(measures[activation].keys())
    plt.subplot(gs[0,actndx])
    for lndx, layerid in enumerate(DO_PLOT_LAYERS):
        #for epoch in epochs:
        #    print('her',epoch, measures[activation][epoch]['MI_XM_upper'])
        xmvalsU = np.array([measures[activation][epoch]['H_M_upper'][layerid] for epoch in epochs])
        if DO_LOWER:
            xmvalsL = np.array([measures[activation][epoch]['H_M_lower'][layerid] for epoch in epochs])
        plt.plot(epochs, xmvalsU, label='Layer %d'%layerid)
        #plt.errorbar(epochs, (xmvalsL + xmvalsU)/2,xmvalsU - xmvalsL, label='Layer %d'%layerid)
    plt.xscale('log')
    plt.yscale('log')
    plt.title(activation)
    plt.ylabel('H(M)')
    
    plt.subplot(gs[1,actndx])
    for lndx, layerid in enumerate(DO_PLOT_LAYERS):
        #for epoch in epochs:
        #    print('her',epoch, measures[activation][epoch]['MI_XM_upper'])
        xmvalsU = np.array([measures[activation][epoch]['MI_XM_upper'][layerid] for epoch in epochs])
        if DO_LOWER:
            xmvalsL = np.array([measures[activation][epoch]['MI_XM_lower'][layerid] for epoch in epochs])
        plt.plot(epochs, xmvalsU, label='Layer %d'%layerid)
        #plt.errorbar(epochs, (xmvalsL + xmvalsU)/2,xmvalsU - xmvalsL, label='Layer %d'%layerid)
    plt.xscale('log')
    plt.ylabel('I(X;M)')


    plt.subplot(gs[2,actndx])
    for lndx, layerid in enumerate(DO_PLOT_LAYERS):
        ymvalsU = np.array([measures[activation][epoch]['MI_YM_upper'][layerid] for epoch in epochs])
        if DO_LOWER:
            ymvalsL = np.array([measures[activation][epoch]['MI_YM_lower'][layerid] for epoch in epochs])
        plt.plot(epochs, ymvalsU, label='Layer %d'%layerid)
    plt.xscale('log')
    plt.ylabel('MI(Y;M)')

    plt.subplot(gs[3,actndx])
    for lndx, layerid in enumerate(DO_PLOT_LAYERS):
        #h2vals = np.array([measures[activation][epoch]['H_M2'][layerid] for epoch in epochs])
        h2vals = np.array([measures[activation][epoch]['MI_XM_bin'][layerid] for epoch in epochs])
        plt.semilogx(epochs, h2vals, label='Layer %d'%layerid)
    plt.xlabel('Epoch')
    #plt.ylabel("H'(M)")
    plt.ylabel("I(X;M)bin")
    #plt.yscale('log')
    
    if actndx == 0:
        plt.legend(loc='lower right')
        
plt.tight_layout()



In [ ]:
#PLOT_LAYERS = [1,2,3]

import seaborn as sns
sns.set_style('darkgrid')

max_epoch = max( (max(measures[activation].keys()) if len(measures[activation]) else 0) for activation, _ in BASE_DIRS)
sm = plt.cm.ScalarMappable(cmap='gnuplot', norm=plt.Normalize(vmin=0, vmax=max_epoch))
sm._A = []

fig=plt.figure(figsize=(10,5))
for actndx, (activation, _) in enumerate(BASE_DIRS):
    epochs = sorted(measures[activation].keys())
    plt.subplot(1,2,actndx+1)    
    #     if actndx == 0:
    #         ax1 = plt.subplot(1,2,actndx+1)
    #     else:
    #         plt.subplot(1,2,actndx+1, sharey=ax1)
    for epoch in epochs:
        c = sm.to_rgba(epoch)
        xmvals = np.array(measures[activation][epoch]['MI_XM_'+infoplane_measure])[DO_PLOT_LAYERS]
        ymvals = np.array(measures[activation][epoch]['MI_YM_'+infoplane_measure])[DO_PLOT_LAYERS]

        plt.plot(xmvals, ymvals, c=c, alpha=0.1, zorder=1)
        plt.scatter(xmvals, ymvals, s=20, facecolors=[c for _ in DO_PLOT_LAYERS], edgecolor='none', zorder=2)
    
    plt.ylim([0, 3.5])
    plt.xlim([0, 14])
    plt.xlabel('I(X;M)')
    plt.ylabel('I(Y;M)')
    plt.title(activation)
    
cbaxes = fig.add_axes([1.0, 0.125, 0.03, 0.8]) 
plt.colorbar(sm, label='Epoch', cax=cbaxes)
plt.tight_layout()

if DO_SAVE:
    plt.savefig('infoplane-%s-%s.pdf'%(ARCH,infoplane_measure),bbox_inches='tight')
    
    
    

In [ ]:
adsfsdfasdf


plt.figure(figsize=(12,5))

gs = gridspec.GridSpec(len(BASE_DIRS), len(PLOT_LAYERS))
for actndx, (activation, BASE_DIR) in enumerate(BASE_DIRS):
    epochs = []
    means = []
    stds = []
    wnorms = []
    if not os.path.exists(BASE_DIR):
        continue
    for epochfile in sorted(os.listdir(BASE_DIR)):
        if not epochfile.startswith('epoch'):
            continue

        epoch =  int(epochfile[len('epoch'):])
        if epoch > MAX_EPOCHS:
            continue

        with open(BASE_DIR+ "/"+epochfile, 'rb') as f:
            d = cPickle.load(f)
            
        epochs.append(epoch)
        wnorms.append(d['data']['weights_norm'])
        means.append(d['data']['gradmean'])
        stds.append(d['data']['gradstd'])

    wnorms, means, stds = map(np.array, [wnorms, means, stds])
    for lndx,layerid in enumerate(PLOT_LAYERS):
        plt.subplot(gs[actndx, lndx])
        plt.plot(epochs, means[:,layerid], 'b', label="Mean")
        plt.plot(epochs, stds[:,layerid], 'orange', label="Std")
        plt.plot(epochs, means[:,layerid]/stds[:,layerid], 'red', label="SNR")
        plt.plot(epochs, wnorms[:,layerid], 'g', label="||W||")

        #plt.ylabel('Layer %d - SNR'%rowndx)
        #plt.subplot(gs[rowndx*2+1,colndx])
        #plt.plot(epochs, [saved_logs[epoch][t+'_layer_%d_weightsnorm' % rowndx] for epoch in epochs], 'b', label="Weight norm")
        #plt.ylabel('Layer %d - ||Weights||'%rowndx)
        #plt.xlabel('Epochs')
        plt.title('Layer %d'%layerid)
        plt.xlabel('Epoch')
        plt.gca().set_xscale("log", nonposx='clip')
        plt.gca().set_yscale("log", nonposy='clip')
    

plt.legend(loc='lower left')
plt.tight_layout()

In [ ]:
entropy_func_upper = K.function([activityVar,], [kde.entropy_estimator_kl(activityVar, variance),])

plt.figure()
for epoch in (100, 2000):
    with open("rawdata/tanh_20-20-20-20/epoch%08d"%epoch, 'rb') as f:
        d = cPickle.load(f)
        activity = d['data']['activity_tst'][3]
        plt.figure()
        plt.hist(activity.flatten())
        print(epoch, entropy_func_upper([activity,])[0])

# plt.figure()
# with open("rawdata/tanh_1024-20-20-20/epoch%08d"%100, 'rb') as f:
#     d = cPickle.load(f)
#     activity = d['data']['activity_tst'][3]
#     plt.hist(activity.flatten())
    
    

In [ ]:
xmvalsU = np.array([measures['tanh'][epoch]['MI_XM_upper'][layerid] for epoch in epochs])
print(xmvalsU)

In [ ]:
import numpy as np
import kde
X = np.random.random((10000,100))*10
print(np.log(10000))
print(entropy_func_upper([X,]) - kde.kde_condentropy(X, variance))
print(entropy_func_upper([X,]))
d2 = K.function([activityVar,], [kde.Kget_dists(activityVar),])
dists = d2([X,])[0]

from scipy.misc import logsumexp
N, dims = X.shape
#dims, N = get_shape(x)
# dists = Kget_dists(x)
dists2 = dists / (2*variance)
normconst = (dims/2.0)*np.log(2*np.pi*variance)
lprobs = logsumexp(-dists2, axis=1) - np.log(N) - normconst
print(lprobs)
h = -np.mean(lprobs)
v= dims/2 + h
print(v)
print(normconst)


In [ ]:
dims, N = get_shape(x)
val = entropy_estimator_kl(x,4*var)
return val + np.log(0.25)*dims/2